# RAG Project - General Handbook / Querying Database and Prompting LLM
## Querying Pinecone
I will start by importing the required libraries.

In [3]:
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai

from dotenv import load_dotenv
import os
load_dotenv()

## Initializing connection to Pinecone index
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("general-handbook")

## Initializing Google API to generate embeddings
google_api_key=os.environ.get("GOOGLE_API_KEY")

genai.configure(api_key=google_api_key)

/home/angelo/ai-internship/personal-ai-project/.venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Now, I will create a function that sends queries to Pinecone and returns the response.

In [6]:
def query_pinecone(query_text, top_k=5):
    """
    This function queries the Pinecone index for the given query text and returns the top-k results as a list of formatted strings.
    args:
        query_text: string
        top_k: integer
    """
    
    query_vector = genai.embed_content(
        model = "models/embedding-001",
        content = query_text,
        task_type = "retrieval_document",
        title = "Embedding of single string"
    )

    results = index.query(
        namespace = "general-handbook-vectors",
        vector = query_vector["embedding"],
        top_k = top_k,
        include_values = False,
        include_metadata = True,
        # filter = {"genre": {"$eq": "action"}}
    )

    match_list = []

    for matched in results["matches"]:
        match_string = f"""
        Chapter: {matched['metadata']['chapter']}
        Header: {matched['metadata']['title']}
        Section: {matched['metadata']['section']}
        Url: {matched['metadata']['url']}
        --------------------------------------------------
        Content: {matched['metadata']['content']}
        """

        match_list.append(match_string)

        match_string = "\n".join(match_list)

    return match_string

In [7]:
user_input = "What are the questions for the temple recommend interview?"

match_string = query_pinecone(user_input, top_k = 5)

print(match_string)


        Chapter: Temple Recommends
        Header: Conducting a Temple Recommend Interview
        Section: 26.3
        Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/26-temple-recommends?lang=eng
        --------------------------------------------------
        Content: The temple is the house of the Lord. Entering the temple and participating in ordinances there is a sacred privilege. This privilege is reserved for those who are spiritually prepared and striving to live the Lord’s standards, as determined by authorized priesthood leaders. To make this determination, priesthood leaders interview the member using the questions below. Leaders should not add or remove any requirements. However, they may adapt the questions to the age and circumstances of the member. Sometimes members have questions during a temple recommend interview. The priesthood leader may explain basic gospel principles. He may also help members understand the temple recommend questions if

## Prompting Gemini with queried vectors
Now, I will create a function that takes up the results from the query to Pinecone, and uses them to answer the user's question.

First, let's import and create the necessary funcitons for the initial setup.

In [10]:
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

text_generator = genai.GenerativeModel('gemini-1.5-flash')

I will first try with a test prompt, and adjust it before creating the funtions that will put it all together.

In [11]:
promt = f"""
Your role:
You are a chatbot that answers questions about the General Handbook of the Church of Jesus Christ of Latter Day Saints.

Intructions:
Below is a list of information received from a vector database where you might find information to answer the user's question about the General Handbook.
Each chunk of information contains a title, url, and content. You will mainly answer questions from the content, but try also to include references for the user.

Information:
START OF INFORMATION

{match_string}
END OF INFORMATION

(Further instructions: If the user seems to be asking about something not related to the General Handbook, invite them to ask about it. If instead of a questions they seem to be thanking you for previous answers, show that you welcome their gratitude)
User Input or question:

{user_input}
"""
print(promt)


Your role:
You are a chatbot that answers questions about the General Handbook of the Church of Jesus Christ of Latter Day Saints.

Intructions:
Below is a list of information received from a vector database where you might find information to answer the user's question about the General Handbook.
Each chunk of information contains a title, url, and content. You will mainly answer questions from the content, but try also to include references for the user.

Information:
START OF INFORMATION


        Chapter: Temple Recommends
        Header: Conducting a Temple Recommend Interview
        Section: 26.3
        Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/26-temple-recommends?lang=eng
        --------------------------------------------------
        Content: The temple is the house of the Lord. Entering the temple and participating in ordinances there is a sacred privilege. This privilege is reserved for those who are spiritually prepared and striving to liv

In [12]:
response = text_generator.generate_content(promt)

to_markdown(response.text)

> The General Handbook doesn't specify the questions for the temple recommend interview. It states that authorized priesthood leaders determine if a member is spiritually prepared to enter the temple by interviewing them using pre-defined questions. You can find more details in the General Handbook section 26.3: [https://www.churchofjesuschrist.org/study/manual/general-handbook/26-temple-recommends?lang=eng](https://www.churchofjesuschrist.org/study/manual/general-handbook/26-temple-recommends?lang=eng) 


Finally, I will use gradio as my UI, and I will use the query_pinecone function and the generate_content function to prompt Gemini for answers to the users questions.

In [ ]:
import random
import gradio as gr

def format_chat_prompt(message, chat_history):
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history):
        
        #information = query_pinecone(message)        
        information = query_pinecone(message, top_k=7)
        print(information)

        full_prompt = f"""
        Your role:
        You are a chatbot that answers questions about the General Handbook of the Church of Jesus Christ of Latter Day Saints.

        Intructions:
        Below is a list of information received from a vector database where you might find information to answer the user's question about the General Handbook.
        Each chunk of information contains a title, url, and content. You will mainly answer questions from the content, but feel free to share that extra information as reference.

        Information:
        START OF INFORMATION

        {information}

        END OF INFORMATION

        (Further instructions: If the user seems to be asking about something not related to the General Handbook, invite them to ask about it. If instead of a questions they seem to be thanking you for previous answers, show that you welcome their gratitude.)
        User Input:
        {message}
        """


        formatted_prompt = format_chat_prompt(full_prompt, chat_history)
        bot_message = text_generator.generate_content(formatted_prompt).text
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Question")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch(share=True)